# 2021/12/27 定期勉強会 データ分析演習

#### 使用データ：[House prices](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

## ライブラリの読み込み

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as sp

## データの読み込み

In [ ]:
data =  pd.read_csv('https://raw.githubusercontent.com/e4rll/FUseminer_20211227/master/LR_sampleData.csv')

In [ ]:
data.head()

## 仮説の確認
<br>
仮説：住宅価格(\$)は、「敷地面積(feet$^2$)」、「建築年・改築年からの経過年数(年)」、「地上以上の部屋数(部屋)」によって算出される。

## 使用する変数
<br>
・目的変数（予測したい値）

SalePrice - 住宅価格
<br><br>

・説明変数（予測したい値に影響を与える変数）

LotArea - 敷地面積  

YearRemodAdd - 建築年・改築年

TotRmsAbvGrd - 地上以上の部屋数

YrSold - 販売年

## データ収集
<br>
仮説に沿ったデータのみを抽出する。

In [ ]:
data = data[['SalePrice', 'LotArea', 'YearRemodAdd', 'TotRmsAbvGrd', 'YrSold']]

### データの確認
<br>
変数の数とデータ件数の確認。

In [ ]:
print('(行数, 列数) = ', data.shape)

抽出したデータのみのデータ確認。

In [ ]:
data.head()

データの情報を確認する。

In [ ]:
data.info()

### 欠損値の確認

In [ ]:
print('Train columns with null values: {} \n' .format(data.isnull().sum()))

### データの基礎統計量の確認
<br>
・項目の説明

count: 要素の個数

mean: 平均

std: 標準偏差

min: 最小値

25%: 第一四分位数

75%: 第三四分位数

50%: 中央値 (median)

max: 最大値

In [ ]:
data.describe()

相関係数の算出

In [ ]:
corr = corr = data.corr()
display(corr)

相関係数を見やすく。

In [ ]:
sns.set_context('talk')
fig = plt.subplots(figsize = (8, 8))
 
sns.heatmap(corr, annot = True, fmt = '.2f', cmap = 'Blues', square = True)

全体のヒストグラムと散布図の描画。

In [ ]:
sns.pairplot(data)

### 目的変数のみ確認

ヒストグラム描画

In [ ]:
%matplotlib inline
data['SalePrice'].hist(bins = 50, figsize = (10, 7.5))
plt.rcParams['figure.figsize'] = [20, 15]

箱ひげ図描画

In [ ]:
%matplotlib inline
fig_o, ax_o = plt.subplots(figsize=(9.6, 5))
ax_o.set_title('Box Plot')
ax_o.boxplot(data['SalePrice'], vert = False,  labels=['SalePrice'])
plt.rcParams['figure.figsize'] = [20, 15]

目的変数が正規分布に従っていないことを確認しておく。

### 説明変数のみ確認

ヒストグラム描画

In [ ]:
%matplotlib inline
data[['LotArea', 'YearRemodAdd', 'TotRmsAbvGrd', 'YrSold']].hist(bins=50, figsize=(15,7.5))
plt.rcParams['figure.figsize'] = [20, 15]

## データ加工

### 変数の作成
<br>
仮説では、「建築年・改築年からの経過年数(年)」が住宅価格に影響を与えているとしていたので、「経過年数」という変数を作成する。

それに伴い、元データの「建築年・改築年」、「販売年」という変数を削除する。

In [ ]:
data['ElapYr'] = data['YrSold'] - data['YearRemodAdd']
data.drop(['YearRemodAdd', 'YrSold'], axis = 1, inplace = True)

データの再確認

In [ ]:
data.head()

In [ ]:
corr = corr = data.corr()
display(corr)

In [ ]:
sns.set_context('talk')
fig = plt.subplots(figsize = (8, 8))
 
sns.heatmap(corr, annot = True, fmt = '.2f', cmap = 'Blues', square = True)

In [ ]:
sns.pairplot(data)

### 外れ値対処

In [ ]:
%matplotlib inline
fig_o, ax_o = plt.subplots(figsize=(9.6, 5))
ax_o.set_title('Box Plot')
ax_o.boxplot(data['LotArea'], vert = False,  labels=['LotArea'])
plt.rcParams['figure.figsize'] = [20, 15]

外れ値の除去。

In [ ]:
data.query('LotArea <= 39000', inplace = True)

In [ ]:
%matplotlib inline
data['LotArea'].hist(bins = 50, figsize = (10, 7.5))
plt.rcParams['figure.figsize'] = [20, 15]

In [ ]:
corr = corr = data.corr()
display(corr)

In [ ]:
sns.set_context('talk')
fig = plt.subplots(figsize = (8, 8))
 
sns.heatmap(corr, annot = True, fmt = '.2f', cmap = 'Blues', square = True)

「敷地面積」の外れ値を除去したら、「敷地面積」と「地上以上の部屋数」間に多重共線性(マルチコ)がありそうに見えてきましたが、今回は取り敢えず無視します。

In [ ]:
sns.pairplot(data)

## 回帰分析

一旦データを再確認する。

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
%matplotlib inline
np.log(data['SalePrice']).hist(bins = 50, figsize = (10, 7.5))
plt.rcParams['figure.figsize'] = [20, 15]

### モデル構築
<br>
・目的変数：

SalePrice - 住宅価格
<br><br>
・説明変数：

LotArea - 敷地面積

TotRmsAbvGrd - 地上以上の部屋数

ElapYr - 経過年数
<br><br>
#### モデル式
$ {\rm SalePrice} = \beta_0 + {\rm LotArea} \: \times \beta_1 + {\rm TotRmsAbvGrd} \: \times \beta_2 + {\rm ElapYr} \: \times \beta_3 $

In [ ]:
x = data[['LotArea', 'TotRmsAbvGrd', 'ElapYr']]
y = np.log(data['SalePrice'])
X = sm.add_constant(x)

model = sm.OLS(y, X)
result = model.fit()
result.summary()

### 着目する値
<br>
・Adj. R-squared: 自由度調整済み決定係数

・Prob (F-statistic): モデルに対するF検定の有意確率

・coef: 偏回帰係数

・P>|t|: 偏回帰係数に対するt検定の有意確率

・（必要に応じて）AIC: 赤池情報量基準

・（必要に応じて）BIC: ベイズ情報量基準

## 影響度比較

In [ ]:
x = data[['LotArea', 'TotRmsAbvGrd', 'ElapYr']]
y = np.log(data['SalePrice'])
y.reset_index(inplace=True, drop=True)
x_n = pd.DataFrame(sp.stats.zscore(x, axis=0), \
                   columns=['LotArea', 'TotRmsAbvGrd', 'ElapYr'])
X_n = sm.add_constant(x_n)

model = sm.OLS(y, X_n)
result = model.fit()
result.summary()